<a href="https://colab.research.google.com/github/yalem123/Amharic-E-commerce-Data-Extractor/blob/main/Copy_of_Telegram_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install telethon

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.0/722.0 kB 11.5 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=ce16e5f773af2c34c3ce9a3daff3b0acecffc9b4666e3b83c71ddfd27df7f225
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


In [ ]:
# Always run first
!pip install -q torch==2.0.1 transformers==4.30.0 datasets==2.12.0
import torch
torch.cuda.empty_cache()  # Clear GPU memory

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 750.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install telethon pandas python-dotenv

In [ ]:
# Create the .env file (RUN THIS FIRST)
with open('/content/.env', 'w') as f:
    f.write("""API_ID=25991276
API_HASH=e3d945cb43e690489f6dbda63cd236e8
PHONE=+251923462419""")

# Verify it was created
print("File created at: /content/.env")
print("Contents:")
!cat /content/.env

In [ ]:
# Confirm the file exists
!ls -la /content/.env

# Expected output:
# -rw-r--r-- 1 root root 83 Jun 20 12:34 /content/.env

-rw-r--r-- 1 root root 77 Jun 23 06:20 /content/.env


In [ ]:
from dotenv import load_dotenv
import os

# FORCE reload with exact path
load_dotenv('/content/.env', override=True, verbose=True)  # ← See loading debug

# Nuclear verification
print("\n🔥 DIRECT FILE READ:")
with open('/content/.env') as f:
    print(f.read())

print("\n🔥 OS.ENV OUTPUT:")
print("API_ID:", os.getenv('API_ID', 'NOT FOUND!'))
print("API_HASH:", os.getenv('API_HASH', 'NOT FOUND!'))
print("PHONE:", os.getenv('PHONE', 'NOT FOUND!'))

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv('/content/.env')  # Load the .env file

api_id = int(os.getenv('API_ID'))
api_hash = os.getenv('API_HASH')
phone = os.getenv('PHONE')

print("✅ Credentials loaded successfully:")
print(f"API_ID: {api_id} (type: {type(api_id)})")
print(f"API_HASH: {api_hash}")
print(f"PHONE: {phone}")

In [ ]:
async def scrape_channel(channel_name, limit=50):
    async with TelegramClient('ethiomart_session', api_id, api_hash) as client:
        await client.start(phone)
        print(f"🔄 Scraping {channel_name}...")

        all_messages = []
        offset_id = 0

        while len(all_messages) < limit:
            try:
                # Updated GetHistoryRequest with all required parameters
                batch = await client(GetHistoryRequest(
                    peer=channel_name,
                    limit=min(100, limit - len(all_messages)),
                    offset_date=None,     # New required
                    offset_id=offset_id,
                    max_id=0,             # New required
                    min_id=0,            # New required
                    add_offset=0,         # New required
                    hash=0                # New required
                ))

                if not batch.messages:
                    break

                for msg in batch.messages:
                    if hasattr(msg, 'message') and msg.message:
                        all_messages.append({
                            'date': msg.date,
                            'message': msg.message,
                            'views': msg.views if hasattr(msg, 'views') else 0,
                            'channel': channel_name,
                            'post_id': msg.id
                        })

                offset_id = batch.messages[-1].id
                print(f"📥 Collected {len(all_messages)}/{limit} messages...")

            except Exception as e:
                print(f"⚠️ Error: {str(e)}")
                break

        return pd.DataFrame(all_messages)

# Test again
test_data = await scrape_channel('ZemenExpress', limit=20)
test_data.head()

🔄 Scraping ZemenExpress...
📥 Collected 7/20 messages...
📥 Collected 14/20 messages...
📥 Collected 15/20 messages...
📥 Collected 18/20 messages...
📥 Collected 19/20 messages...
📥 Collected 19/20 messages...
📥 Collected 19/20 messages...
📥 Collected 19/20 messages...
📥 Collected 19/20 messages...
📥 Collected 20/20 messages...


,date,message,views,channel,post_id
0,2025-06-21 16:35:51+00:00,💥💥...................................💥💥\n\n📌Sa...,2293,ZemenExpress,6991
1,2025-06-21 08:07:31+00:00,💥💥...................................💥💥\n\n3pc...,2660,ZemenExpress,6987
2,2025-06-21 08:07:11+00:00,💥💥...................................💥💥\n\n3pc...,2269,ZemenExpress,6986
3,2025-06-21 05:42:46+00:00,💥💥...................................💥💥\n\n📌1 ...,2402,ZemenExpress,6985
4,2025-06-21 05:42:19+00:00,💥💥...................................💥💥\n\n📌1 ...,2561,ZemenExpress,6983


In [ ]:
target_channels = [
    'ZemenExpress',
    'helloomarketethiopia',
    'Fashiontera',
    'modernshoppingcenter',
    'Shewabrand'
]

async def scrape_all_channels():
    all_data = []
    for channel in target_channels:
        try:
            print(f"\n📡 Scraping {channel}...")
            channel_data = await scrape_channel(channel, limit=100)  # 100 msgs/channel
            all_data.append(channel_data)
            print(f"✅ Got {len(channel_data)} messages from {channel}")
        except Exception as e:
            print(f"❌ Failed on {channel}: {str(e)}")
    return pd.concat(all_data)

# Run (after fixing GetHistoryRequest as previously shown)
full_data = await scrape_all_channels()
print(f"\n🌟 TOTAL MESSAGES: {len(full_data)}")
full_data.to_csv('all_channels_raw.csv', index=False)


📡 Scraping ZemenExpress...
🔄 Scraping ZemenExpress...
📥 Collected 40/100 messages...
📥 Collected 64/100 messages...
📥 Collected 77/100 messages...
📥 Collected 88/100 messages...
📥 Collected 91/100 messages...
📥 Collected 95/100 messages...
📥 Collected 96/100 messages...
📥 Collected 98/100 messages...
📥 Collected 98/100 messages...
📥 Collected 99/100 messages...
📥 Collected 100/100 messages...
✅ Got 100 messages from ZemenExpress

📡 Scraping helloomarketethiopia...
🔄 Scraping helloomarketethiopia...
📥 Collected 76/100 messages...
📥 Collected 100/100 messages...
✅ Got 100 messages from helloomarketethiopia

📡 Scraping Fashiontera...
🔄 Scraping Fashiontera...
📥 Collected 41/100 messages...
📥 Collected 63/100 messages...
📥 Collected 77/100 messages...
📥 Collected 85/100 messages...
📥 Collected 91/100 messages...
📥 Collected 96/100 messages...
📥 Collected 98/100 messages...
📥 Collected 99/100 messages...
📥 Collected 100/100 messages...
✅ Got 100 messages from Fashiontera

📡 Scraping modern

In [ ]:
import re  # <-- ADD THIS IMPORT AT THE TOP
import pandas as pd

def clean_amharic_commerce(text):
    """Enhanced cleaner for Ethiopian e-commerce text"""
    if not isinstance(text, str):
        return ""

    try:
        # 1. Remove URLs, mentions, and phone numbers
        text = re.sub(
            r'http\S+|www\S+|@\w+|[\+0]9\d{8}',  # Ethio phone numbers
            '',
            text
        )

        # 2. Standardize Ethiopian currency (ብር, Br, etc.)
        text = re.sub(r'(\d+)\s*(ብር|br|Br|birr|BIRR)', r'\1 ብር', text, flags=re.IGNORECASE)

        # 3. Keep only Amharic + basic punctuation
        text = re.sub(r'[^\w\s\u1200-\u137F.,!?]', '', text)

        # 4. Normalize whitespace and return
        return ' '.join(text.split()).strip()

    except Exception as e:
        print(f"Error cleaning text: {str(e)}")
        return ""

# Apply cleaning with progress bar
from tqdm import tqdm
tqdm.pandas()

full_data['clean_text'] = full_data['message'].progress_apply(clean_amharic_commerce)
full_data = full_data[full_data['clean_text'].str.len() > 5]  # Remove empty/short texts

# Save cleaned data
full_data.to_csv('all_channels_clean.csv', index=False)
print(f"✅ Cleaned {len(full_data)} messages. Sample:")
print(full_data[['channel', 'clean_text']].head(3))

100%|██████████| 500/500 [00:00<00:00, 14865.93it/s]

✅ Cleaned 499 messages. Sample:
        channel                                         clean_text
0  ZemenExpress  ................................... Saachi Ele...
1  ZemenExpress  ................................... 3pcs Bottl...
2  ZemenExpress  ................................... 3pcs Bottl...


In [ ]:
import re
import pandas as pd

def clean_amharic_commerce(text):
    """Properly preserves Amharic text while cleaning"""
    if not isinstance(text, str):
        return ""

    # Keep Amharic + basic Latin + numbers + punctuation
    text = re.sub(
        r'[^\w\s\u1200-\u137F.,!?]',  # Amharic Unicode range: \u1200-\u137F
        '',
        text
    )

    # Standardize price formats (e.g., "1000ብር" → "1000 ብር")
    text = re.sub(r'(\d+)([^\s\d])', r'\1 \2', text)

    return ' '.join(text.split()).strip()

# Re-apply cleaning
full_data['clean_text'] = full_data['message'].apply(clean_amharic_commerce)
full_data = full_data[full_data['clean_text'].str.len() > 5]

# Verify Amharic is preserved
print("✅ Corrected cleaning. Sample:")
print(full_data['clean_text'].head(3).to_markdown())

✅ Corrected cleaning. Sample:
|    | clean_text                                                                                                                                                                                                                                                                                                                                                                      |
|---:|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | ................................... Saachi Electric Kettle Borosilicate Glass Body Overheat protection Automatic switch off 2200 w ዋጋ፦ 2700 ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05 S06 09026607

In [ ]:
import re

def clean_amharic_commerce(text):
    """Properly preserves Amharic and English text"""
    if not isinstance(text, str):
        return ""

    # Keep Amharic, English, numbers, and basic punctuation
    text = re.sub(
        r'[^\w\s\u1200-\u137F.,!?]',  # Amharic range + basic chars
        ' ',  # Replace unwanted chars with space (not dots)
        text
    )

    # Standardize price formats
    text = re.sub(r'(\d+)\s*(ብር|br|birr)?', r'\1 ብር', text, flags=re.IGNORECASE)

    # Remove multiple spaces
    return ' '.join(text.split())

# Test with your sample
sample = "Saachi Electric Kettle 2700 ብር አድራሻ፦ መገናኛ_መሰረት http://t.me/zemenexpress"
print(clean_amharic_commerce(sample))

Saachi Electric Kettle 2700 ብር አድራሻ፦ መገናኛ_መሰረት http t.me zemenexpress


In [ ]:
async def scrape_channel(channel_name, limit=100):
    async with TelegramClient('ethiomart_session', api_id, api_hash) as client:
        await client.start(phone)

        try:
            # First resolve the channel entity
            channel = await client.get_entity(channel_name)

            all_messages = []
            offset_id = 0

            while len(all_messages) < limit:
                batch = await client(GetHistoryRequest(
                    peer=channel,
                    limit=min(100, limit - len(all_messages)),
                    offset_id=offset_id,
                    offset_date=None,
                    add_offset=0,
                    max_id=0,
                    min_id=0,
                    hash=0
                ))

                if not batch.messages:
                    break

                for msg in batch.messages:
                    if hasattr(msg, 'message') and msg.message:
                        all_messages.append({
                            'date': msg.date,
                            'message': msg.message,
                            'views': msg.views if hasattr(msg, 'views') else 0
                        })

                offset_id = batch.messages[-1].id
                print(f"Collected {len(all_messages)}/{limit}")

            return pd.DataFrame(all_messages)

        except Exception as e:
            print(f"Error in {channel_name}: {str(e)}")
            return pd.DataFrame()

In [ ]:
full_data['clean_text'] = full_data['message'].apply(clean_amharic_commerce)
print(full_data[['clean_text']].head())

                                          clean_text
0  ................................... Saachi Ele...
1  ................................... 3 ብርpcs Bo...
2  ................................... 3 ብርpcs Bo...
3  ................................... 1 ብርpairs ...
4  ................................... 1 ብርpairs ...


In [ ]:
test_df = await scrape_channel('ZemenExpress', limit=20)

Collected 7/20
Collected 14/20
Collected 15/20
Collected 18/20
Collected 19/20
Collected 19/20
Collected 19/20
Collected 19/20
Collected 19/20
Collected 20/20


In [ ]:
# Check the scraped data
print(f"✅ Successfully collected {len(test_df)} messages:")
print(test_df[['date', 'message', 'views']].head(3))

# Save raw data
test_df.to_csv('zemenexpress_raw.csv', index=False)
!cp zemenexpress_raw.csv '/content/drive/MyDrive/ethiomart_data/'  # Backup to Google Drive

✅ Successfully collected 20 messages:
                       date  \
0 2025-06-21 16:35:51+00:00   
1 2025-06-21 08:07:31+00:00   
2 2025-06-21 08:07:11+00:00   

                                             message  views  
0  💥💥...................................💥💥\n\n📌Sa...   2310  
1  💥💥...................................💥💥\n\n3pc...   2671  
2  💥💥...................................💥💥\n\n3pc...   2276  
cp: cannot create regular file '/content/drive/MyDrive/ethiomart_data/': No such file or directory


In [ ]:
# Apply our improved cleaner
test_df['clean_text'] = test_df['message'].apply(clean_amharic_commerce)

# Verify cleaning worked
print("\n🧹 Cleaned text samples:")
print(test_df[['message', 'clean_text']].head(3).to_markdown())


🧹 Cleaned text samples:
|    | message                                     | clean_text                                                                                                                                                                                                                                                                                                                                                                                    |
|---:|:--------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | 💥💥...................................💥💥 | ................................... Saachi Electric 

In [ ]:
# Select 15 diverse samples for labeling
label_sample = test_df.sample(15, random_state=42)[['clean_text']]

# Save for Label Studio
label_sample.to_csv('ner_labeling_samples.csv', index=False)
print(f"\n✏️ Label these 15 messages in 'ner_labeling_samples.csv'")


✏️ Label these 15 messages in 'ner_labeling_samples.csv'


In [ ]:
channels = ['helloomarketethiopia', 'Fashiontera', 'modernshoppingcenter', 'Shewabrand']
all_data = [test_df]  # Start with ZemenExpress data

for channel in channels:
    try:
        channel_data = await scrape_channel(channel, limit=50)
        channel_data['clean_text'] = channel_data['message'].apply(clean_amharic_commerce)
        all_data.append(channel_data)
        print(f"✅ {channel}: {len(channel_data)} messages")
    except Exception as e:
        print(f"❌ {channel}: {str(e)}")

full_data = pd.concat(all_data)
full_data.to_csv('all_channels_clean.csv', index=False)
print(f"\n🌟 Total messages: {len(full_data)}")

Collected 36/50
Collected 43/50
Collected 48/50
Collected 49/50
Collected 50/50
✅ helloomarketethiopia: 50 messages
Collected 24/50
Collected 36/50
Collected 38/50
Collected 43/50
Collected 45/50
Collected 46/50
Collected 48/50
Collected 49/50
Collected 49/50
Collected 50/50
✅ Fashiontera: 50 messages
Collected 12/50
Collected 21/50
Collected 29/50
Collected 34/50
Collected 37/50
Collected 41/50
Collected 43/50
Collected 45/50
Collected 46/50
Collected 48/50
Collected 49/50
Collected 49/50
Collected 49/50
Collected 49/50
Collected 50/50
✅ modernshoppingcenter: 50 messages
Collected 39/50
Collected 50/50
✅ Shewabrand: 50 messages

🌟 Total messages: 220


In [ ]:
print("Columns in your DataFrame:", full_data.columns.tolist())

Columns in your DataFrame: ['date', 'message', 'views', 'clean_text']


In [ ]:
# Create a channel mapping based on message patterns
def detect_channel(message):
    if 'zemen' in message.lower():
        return 'ZemenExpress'
    elif 'helloo' in message.lower():
        return 'helloomarketethiopia'
    elif 'fashion' in message.lower():
        return 'Fashiontera'
    elif 'modern' in message.lower():
        return 'modernshoppingcenter'
    elif 'shewa' in message.lower():
        return 'Shewabrand'
    else:
        return 'unknown'

full_data['channel'] = full_data['message'].apply(detect_channel)

# Verify
print(full_data['channel'].value_counts())

channel
Fashiontera             51
helloomarketethiopia    48
Shewabrand              48
modernshoppingcenter    48
ZemenExpress            20
unknown                  5
Name: count, dtype: int64


In [ ]:
print(f"✅ Channel distribution:\n{full_data['channel'].value_counts()}")
print(f"\n🔍 Sample message from ZemenExpress:")
print(full_data[full_data['channel']=='ZemenExpress']['clean_text'].iloc[0][:100] + "...")

✅ Channel distribution:
channel
Fashiontera             51
helloomarketethiopia    48
Shewabrand              48
modernshoppingcenter    48
ZemenExpress            20
unknown                  5
Name: count, dtype: int64

🔍 Sample message from ZemenExpress:
................................... Saachi Electric Kettle Borosilicate Glass Body Overheat protecti...


In [ ]:
# Take 20 messages from each channel for balanced labeling
balanced_samples = []
for channel in ['ZemenExpress', 'helloomarketethiopia', 'Fashiontera', 'modernshoppingcenter', 'Shewabrand']:
    samples = full_data[full_data['channel']==channel].sample(20, random_state=42)
    balanced_samples.append(samples)

balanced_data = pd.concat(balanced_samples)

In [ ]:
# Create labeling template
labeling_template = balanced_data[['channel', 'clean_text']].reset_index(drop=True)
labeling_template.to_csv('ner_labeling_template.csv', index=False)

print("✏️ Label these columns in your tool:")
print(labeling_template.head(3).to_markdown())

✏️ Label these columns in your tool:
|    | channel      | clean_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|---:|:-------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Auto-detect potential entities to speed up labeling
def prelabel_hints(text):
    hints = []
    # Price detection
    if re.search(r'\d+\s*ብር', text):
        hints.append(("PRICE", re.findall(r'\d+\s*ብር', text)))
    # Location hints
    ethio_cities = ['አዲስ አበባ', 'ቦሌ', 'ሀዋሳ', 'ድሬ ዳዋ']
    if any(city in text for city in ethio_cities):
        hints.append(("LOCATION", [city for city in ethio_cities if city in text]))
    return hints

# Apply to first 5 samples
for i in range(5):
    sample = labeling_template['clean_text'].iloc[i]
    print(f"\nSample {i+1}:")
    print(sample)
    print("Suggested labels:", prelabel_hints(sample))


Sample 1:
................................... Saachi Electric Kettle Borosilicate Glass Body Overheat protection Automatic switch off 2200 ብርw ዋጋ፦ 2700 ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05 ብርS06 ብር0902660722 ብር0928460606 ብርበTelegram ለማዘዝ ይጠቀሙ zemencallcenter zemenexpressadmin ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን https telegram.me zemenexpress ...................................
Suggested labels: [('PRICE', ['2200 ብር', '2700 ብር', '05 ብር', '06 ብር', '0902660722 ብር', '0928460606 ብር'])]

Sample 2:
................................... Shock and Noise Cancelling Washing Machine Support Washer and Dryer Pedestals,Anti Slip Anti Vibration Rubber Washing Machine Feet Pads Washing Machine Stabilizer for Raise Height Reduce Noise 4 ብርPCS ዋጋ፦ 500 ብር ውስን ፍሬ ነው ያለን አድራሻ መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05 ብርS06 ብር0902660722 ብር0928460606 ብርበTelegram ለማዘዝ ይጠቀሙ zemencallcenter zemenexpressadmin ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን https telegram.me zemenexpress ...................................
Suggested lab

In [ ]:
# Basic stats while waiting for labeling
vendor_stats = (full_data.groupby('channel')
                .agg(avg_views=('views', 'mean'),
                     post_count=('message', 'count'))
                .sort_values('avg_views', ascending=False))

print("📈 Top Performing Channels:")
print(vendor_stats)

📈 Top Performing Channels:
                         avg_views  post_count
channel                                       
modernshoppingcenter  10272.229167          48
Shewabrand             7616.625000          48
unknown                4724.600000           5
ZemenExpress           3431.700000          20
Fashiontera            3204.196078          51
helloomarketethiopia   2850.250000          48


In [ ]:
print("Before:", full_data['message'].iloc[0])
print("After:", clean_amharic_commerce(full_data['message'].iloc[0]))

Before: 💥💥...................................💥💥

📌Saachi Electric Kettle

👍Borosilicate Glass Body
👍Overheat protection
👍Automatic switch off
👍2200w

ዋጋ፦  💲🏷 2700  ብር ✅

♦️ውስን ፍሬ ነው ያለው🔥🔥🔥

🏢 አድራሻ👉

📍♦️#መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05/S06


     💧💧💧💧


    📲 0902660722
    📲 0928460606 

🔖
💬በTelegram ለማዘዝ ⤵️ ይጠቀሙ🔽

@zemencallcenter 

@zemenexpressadmin

ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን⤵️
https://telegram.me/zemenexpress

💥💥...................................💥💥
After: ................................... Saachi Electric Kettle Borosilicate Glass Body Overheat protection Automatic switch off 2200 ብርw ዋጋ፦ 2700 ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05 ብርS06 ብር0902660722 ብር0928460606 ብርበTelegram ለማዘዝ ይጠቀሙ zemencallcenter zemenexpressadmin ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን https telegram.me zemenexpress ...................................


In [ ]:
import re

def clean_amharic_commerce(text):
    """Advanced cleaner for Ethiopian e-commerce posts"""
    if not isinstance(text, str):
        return ""

    # Phase 1: Remove emojis and special chars (preserving Amharic)
    text = re.sub(r'[^\w\s\u1200-\u137F.,!?አ-ፕ]', ' ', text)

    # Phase 2: Standardize prices (handle multiple formats)
    text = re.sub(r'(\d+)\s*([ብርbrBR]|\$|€|£)', r'\1 ብር ', text)

    # Phase 3: Remove phone numbers and usernames
    text = re.sub(r'(\+251|0)\d{8}\b|@\w+', '', text)

    # Phase 4: Clean residual artifacts
    text = re.sub(r'\s+', ' ', text)  # Multiple spaces
    text = re.sub(r'(\.\s*){3,}', ' ', text)  # Dots sequences
    return text.strip()

# Test with your problematic sample
sample = """💥💥...................................💥💥
📌Saachi Electric Kettle
👍Borosilicate Glass Body
👍Overheat protection
👍Automatic switch off
👍2200w
ዋጋ፦  💲🏷 2700  ብር ✅
♦️ውስን ፍሬ ነው ያለው🔥🔥🔥
🏢 አድራሻ👉
📍♦️#መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05/S06
📲 0902660722
📲 0928460606
@zemencallcenter
@zemenexpressadmin
https://telegram.me/zemenexpress"""

print("Before:", sample)
print("\nAfter:", clean_amharic_commerce(sample))

Before: 💥💥...................................💥💥
📌Saachi Electric Kettle
👍Borosilicate Glass Body
👍Overheat protection
👍Automatic switch off
👍2200w
ዋጋ፦  💲🏷 2700  ብር ✅
♦️ውስን ፍሬ ነው ያለው🔥🔥🔥
🏢 አድራሻ👉
📍♦️#መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05/S06
📲 0902660722
📲 0928460606 
@zemencallcenter 
@zemenexpressadmin
https://telegram.me/zemenexpress

After: Saachi Electric Kettle Borosilicate Glass Body Overheat protection Automatic switch off 2200w ዋጋ፦ 2700 ብር ር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05 S06 0902660722 0928460606 zemencallcenter zemenexpressadmin https telegram.me zemenexpress


In [ ]:
# Re-clean your entire dataset
full_data['clean_text'] = full_data['message'].apply(clean_amharic_commerce)

# Verify
print(full_data[['message', 'clean_text']].sample(3).to_markdown())

|    | message                                                                                                    | clean_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|---:|:-----------------------------------------------------------------------------------------------------------|:------------

In [ ]:
# 1. Verify cleaned data quality
print("Cleaned Data Sample:")
print(full_data[['channel', 'clean_text']].sample(5).to_markdown())

# 2. Save the processed data
full_data.to_csv('ethiomart_cleaned_messages.csv', index=False)
!cp ethiomart_cleaned_messages.csv '/content/drive/MyDrive/ethiomart_data/'

Cleaned Data Sample:
|    | channel              | clean_text                                                                                                                                                                                                                                                                                                                                                                                                      |
|---:|:---------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 48 | helloomarketethiopia | እንደ አዲስ እንተዋወቅ? ልዩ እና የተሻለ ነገር ለእናንተ ለማምጣት ስንዘጋጅ ቆይተናል። ቀኑን ያዙት በሚያዝያ 4 ከምን ጊዜውም በላቀ 

In [ ]:
import pandas as pd

# Get the minimum available samples across all channels
min_samples = full_data['channel'].value_counts().min()
sample_size = min(10, min_samples)  # Use 10 or whatever is available

label_samples = []
for channel in full_data['channel'].unique():
    channel_data = full_data[full_data['channel'] == channel]
    samples = channel_data.sample(n=sample_size,
                                random_state=42,
                                replace=len(channel_data) < sample_size)  # Allow replacement if needed
    label_samples.append(samples)

label_df = pd.concat(label_samples)
print(f"✅ Collected {len(label_df)} samples ({sample_size} per channel)")

# Verify
print("\nChannel distribution:")
print(label_df['channel'].value_counts())

✅ Collected 30 samples (5 per channel)

Channel distribution:
channel
ZemenExpress            5
helloomarketethiopia    5
unknown                 5
Fashiontera             5
modernshoppingcenter    5
Shewabrand              5
Name: count, dtype: int64


In [ ]:
from google.colab import files
label_df[['channel', 'clean_text']].to_csv('ethiomart_ner_labeling.csv', index=False)
files.download('ethiomart_ner_labeling.csv')  # Download to your computer

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Convert to CoNLL format
def csv_to_conll(input_csv, output_file):
    df = pd.read_csv(input_csv)
    with open(output_file, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            tokens = row['clean_text'].split()
            tags = row['entity_tags'].split() if pd.notna(row.get('entity_tags')) else ['O']*len(tokens)
            for token, tag in zip(tokens, tags):
                f.write(f"{token}\t{tag}\n")
            f.write("\n")

csv_to_conll('ethiomart_ner_labeling.csv', 'ethiomart_ner.conll')
print("✅ Conversion done! First 3 lines:")
!head -n 3 ethiomart_ner.conll

✅ Conversion done! First 3 lines:
Saachi	O
Electric	O
Kettle	O


In [ ]:
def analyze_labels(conll_file):
    with open(conll_file, encoding='utf-8') as f:
        data = f.read()

    # Count tokens (lines that aren't empty)
    total_tokens = data.count('\n') - data.count('\n\n')

    # Count entities
    entities = {}
    for line in data.split('\n'):
        if line.strip() and not line.isspace():
            token, tag = line.split('\t')
            if tag != 'O':
                entities[tag] = entities.get(tag, 0) + 1

    print(f"Total tokens: {total_tokens}")
    print("Entity distribution:")
    for tag, count in entities.items():
        print(f"{count:>5} {tag}")

analyze_labels('ethiomart_ner.conll')

Total tokens: 1582
Entity distribution:


In [ ]:
import pandas as pd
from collections import Counter

def analyze_labels(conll_file):
    # Load CoNLL data
    with open(conll_file, encoding='utf-8') as f:
        lines = [line.strip().split('\t')
                for line in f if line.strip()]

    # Calculate statistics
    total_tokens = len(lines)
    entities = Counter(tag for _, tag in lines)
    o_count = entities.get('O', 0)
    entity_tokens = total_tokens - o_count

    print(f"📝 Token Statistics:")
    print(f"Total tokens: {total_tokens}")
    print(f"Entity tokens: {entity_tokens} ({entity_tokens/total_tokens:.1%})")
    print(f"Non-entity tokens: {o_count} ({o_count/total_tokens:.1%})")

    print("\n🔖 Entity Distribution:")
    for tag, count in sorted(entities.items()):
        if tag != 'O':
            print(f"{tag}: {count:>4} ({count/entity_tokens:.1%})")

# Run analysis
analyze_labels('ethiomart_ner.conll')

📝 Token Statistics:
Total tokens: 1582
Entity tokens: 0 (0.0%)
Non-entity tokens: 1582 (100.0%)

🔖 Entity Distribution:


In [ ]:
import pandas as pd

# Check available columns
label_df = pd.read_csv('ethiomart_ner_labeling.csv')
print("Existing columns:", label_df.columns.tolist())

Existing columns: ['channel', 'clean_text']


In [ ]:
# Option A: If you have labels in another column (e.g., 'labels')
if 'labels' in label_df.columns:
    label_df = label_df.rename(columns={'labels': 'entity_tags'})

# Option B: If starting fresh
else:
    # Initialize all as 'O' (non-entities)
    label_df['entity_tags'] = label_df['clean_text'].apply(
        lambda x: ' '.join(['O']*len(x.split()))
)

# Save corrected version
label_df.to_csv('ethiomart_ner_labeling_FIXED.csv', index=False)
print("✅ Fixed CSV saved. Now add your labels to the 'entity_tags' column!")

✅ Fixed CSV saved. Now add your labels to the 'entity_tags' column!


In [ ]:
!pip install transformers datasets seqeval torch
!pip install sentencepiece  # Required for XLM-Roberta tokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 855.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16250 s

In [ ]:
ls


all_channels_clean.csv          ethiomart_ner_labeling_FIXED.csv
all_channels_raw.csv            ethiomart_session.session
ethiomart_cleaned_messages.csv  ner_labeling_samples.csv
ethiomart_ner_conl.csv          ner_labeling_template.csv
ethiomart_ner.conll             sample_data/
ethiomart_ner_labeling.csv      zemenexpress_raw.csv


In [ ]:
import pandas as pd
import datasets # Import the datasets library

def read_conll(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    tokens, labels = [], []
    current_tokens, current_labels = [], []

    for line in lines:
        line = line.strip()
        if not line:  # Empty line = end of sentence
            if current_tokens:
                tokens.append(current_tokens)
                labels.append(current_labels)
                current_tokens, current_labels = [], []
        else:
            parts = line.split()  # Split by whitespace
            current_tokens.append(parts[0])  # Token (first column)
            current_labels.append(parts[-1]) # Label (last column)

    return {"tokens": tokens, "ner_tags": labels}

# Load data
data = read_conll("ethiomart_ner.conll")
# Now datasets is imported and recognized
dataset = datasets.Dataset.from_dict(data)
print(dataset[0])  # Check the first sentence

{'tokens': ['Saachi', 'Electric', 'Kettle', 'Borosilicate', 'Glass', 'Body', 'Overheat', 'protection', 'Automatic', 'switch', 'off', '2200w', 'ዋጋ፦', '2700', 'ብር', 'ር', 'ውስን', 'ፍሬ', 'ነው', 'ያለው', 'አድራሻ', 'መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ', 'ቢሮ', 'ቁ.', 'S05', 'S06', '0902660722', '0928460606', 'በTelegram', 'ለማዘዝ', 'ይጠቀሙ', 'zemencallcenter', 'zemenexpressadmin', 'ለተጨማሪ', 'ማብራሪያ', 'የቴሌግራም', 'ገፃችን', 'https', 'telegram.me', 'zemenexpress'], 'ner_tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [ ]:
import datasets
from datasets import Dataset, ClassLabel, Sequence
import pandas as pd

def read_conll(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    tokens, ner_tags = [], []
    current_tokens, current_labels = [], []

    for line in lines:
        line = line.strip()
        if not line:  # End of sentence
            if current_tokens:
                tokens.append(current_tokens)
                ner_tags.append(current_labels)
                current_tokens, current_labels = [], []
        else:
            parts = line.split()
            current_tokens.append(parts[0])  # Token
            current_labels.append(parts[-1]) # Label

    return {"tokens": tokens, "ner_tags": ner_tags}

# Load data
data = read_conll("ethiomart_ner.conll")
dataset = Dataset.from_dict(data)

# Define label mappings (update with your actual labels)
label_list = sorted(list(set(tag for tags in data["ner_tags"] for tag in tags)))
label2id = {tag: i for i, tag in enumerate(label_list)}
id2label = {i: tag for tag, i in label2id.items()}

print(f"Labels: {label_list}")
print(dataset[0])  # Inspect first example

Labels: ['O']
{'tokens': ['Saachi', 'Electric', 'Kettle', 'Borosilicate', 'Glass', 'Body', 'Overheat', 'protection', 'Automatic', 'switch', 'off', '2200w', 'ዋጋ፦', '2700', 'ብር', 'ር', 'ውስን', 'ፍሬ', 'ነው', 'ያለው', 'አድራሻ', 'መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ', 'ቢሮ', 'ቁ.', 'S05', 'S06', '0902660722', '0928460606', 'በTelegram', 'ለማዘዝ', 'ይጠቀሙ', 'zemencallcenter', 'zemenexpressadmin', 'ለተጨማሪ', 'ማብራሪያ', 'የቴሌግራም', 'ገፃችን', 'https', 'telegram.me', 'zemenexpress'], 'ner_tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:  # Special tokens
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # New word
                label_ids.append(label2id[label[word_idx]])
            else:  # Subword token
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Use a smaller model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Use a smaller model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
